In [1]:
! venv\Scripts\activate.bat
! pip install -r requirements.txt

You should consider upgrading via the 'python -m pip install --upgrade pip' command.



# Import PyRadios & Create instance of RadioBrowser

In [2]:
import numpy as np

from pyradios import RadioBrowser
from geopy.geocoders import Nominatim
from random import shuffle
from collections import OrderedDict
from IPython.display import FileLink

import geo

rb = RadioBrowser() # wrapper for the API at https://www.radio-browser.info/
geolocator = Nominatim(user_agent="TunerTerrace/1.0") # wrapper for https://nominatim.openstreetmap.org/ui/search.html

# Find Country/State/City associated with longitude and latitude

We'll try to find a station in Kathmandu:
27.7172° N, 85.3240° E

Note that geopy accepts all kinds of longitude/latitude formats. N/S  vs. positive/negative, include or don't include degrees are all fine.

In [3]:
cities = geo.get_city_list('+41.9288-72.6481')
for city in cities:
    print(city['city'])

Worcester
Providence
Springfield
Bridgeport
New Haven


# Find radio stations in/near that city

In [4]:
def score_station(sta):
    """
    Assigns a score for how relevant the station is
    to be used for ranking the order stations are listed
    """
    #Right now just using strict as a binary sorter
    #Could add distance or other info later
    relevance_score = sta['strict'] * 1

    return relevance_score


def get_stations_from_city(loc,rb):
    country = loc['countryCode']
    state = loc['region']
    city = loc['city']

    sta_raw_by = dict()
    sta_uuid_by = dict()
    sta_info_by = dict()

    #calls to radio-browser api
    #TODO: can probably get this down to one call by querying all at once and doing a little filtering client-side
    sta_raw_by['country'] = rb.stations_by_countrycode(code=country)
    if len(sta_raw_by['country']) > 1:
        sta_raw_by['state'] = rb.stations_by_state(state=state)
        sta_raw_by['city_tag'] = rb.stations_by_tag(tag=city)
        sta_raw_by['city_name'] = rb.stations_by_name(name=city)

    #Make hashable data structures from uuids
    for level in sta_raw_by:
        sta_uuid_by[level] = {sta['stationuuid'] for sta in sta_raw_by[level]}

    sta_candidate_uuids = sta_uuid_by['city_name'].union(sta_uuid_by['city_tag'])
    sta_result_uuids = sta_candidate_uuids.intersection(sta_uuid_by['country'])
    sta_result_uuids_strict = sta_result_uuids.intersection(sta_uuid_by['state'])

    #make searchable dict out of country-wide station list by labelling with uuid
    sta_raw_info_dict = {sta['stationuuid']: sta for sta in sta_raw_by['country']}

    #select the appropriate station info from the raw info dict and label if they met the requirements for strictness
    sta_info_dict = dict()
    for uuid in iter(sta_result_uuids):
        sta_info_dict[uuid] = sta_raw_info_dict[uuid]

        strict = uuid in sta_result_uuids_strict
        sta_info_dict[uuid]['strict'] = strict

    #TODO: Ghana and Tonga examples fail below, even though it looks like we should still see some of them. Perhaps it's overfiltered? Need to handle less-populace places better

    return sta_info_dict


def get_stations_from_city_list(loc_ls, rb):  
    sta_info = dict()
    for loc in loc_ls:
        loc_sta_info = get_stations_from_city(loc,rb)
        sta_info.update(loc_sta_info)

    for uuid in sta_info:
        sta = sta_info[uuid]
        sta_info[uuid]['relevance_score'] = score_station(sta)

    sta_finalists = sorted(sta_info.values(), key=lambda value: value['relevance_score'], reverse=True)

    #Filter out low-relevance results. This is so easy with Pandas - use Pandas
    if len(sta_finalists) > 3:
        scores = []
        for sta in sta_finalists:
            scores.append(sta['relevance_score'])
        min_limit = np.max(scores) - np.std(scores)
        sta_display = list(
                            filter(lambda station: station['relevance_score'] >= min_limit,
                                   sta_finalists
                          ))
    else:
        sta_display = sta_finalists

    return sta_display
    


# Pick a station at random from the list

In [5]:
def pick_play_station(choice_ls):
    """
    If multiple stations are tied for highest relevance
    pick a random one and move it to the top of the list
    """
    #This implementation sucks. Use Pandas for all these data structures
    max_score = choice_ls[0]['relevance_score']
    options = [choice_ls.pop(0)]
    while len(choice_ls) and choice_ls[0]['relevance_score'] == max_score:
        options.append(choice_ls.pop(0))
    shuffle(options)

    choice_ls = options + choice_ls
    return choice_ls

def display_stations(choice_ls):
    for sta in choice_ls:
        print(sta['name'] + ' @:')
        print(sta['url'])
        print('Reception: ' + str(sta['relevance_score'] * 100) + '%')
        print('-----')

# Put it all together...

In [18]:
def show_stations_from_coords(coords):
    loc_ls = geo.get_city_list(coords)
    print(f'Found {len(loc_ls)} cities: {", ".join([city["name"] for city in loc_ls])}')
    print()
    choice_ls = get_stations_from_city_list(loc_ls, rb)
    shuffled_choice_ls = pick_play_station(choice_ls)
    display_stations(shuffled_choice_ls)

In [15]:
show_stations_from_coords('+41.9288-72.6481')

Found 5 cities: Worcester,Providence,Springfield,Bridgeport,New Haven
WORC-AM "La Mega" 1310 & 106.1 Worcester, MA @:
http://146.71.118.220:35046/stream
Reception: 100%
-----
WCUW 91.3 Clark University - Worcester, MA @:
http://peridot.streamguys.com:5490/live
Reception: 100%
-----
WCHC @:
https://s2.radio.co/sc161fe4c9/listen
Reception: 100%
-----
WWFX 100.1 "The Pike 100" Southbridge, MA @:
http://provisioning.streamtheworld.com/pls/WWFXFMAAC.pls
Reception: 100%
-----
WPKN Radio 89.5 Bridgeport, CT @:
http://streamcore.elasticbeanstalk.com/wpknhigh.mp3
Reception: 100%
-----
WXLO "104.5 XLO" Fitchburg, MA @:
http://provisioning.streamtheworld.com/pls/WXLOFMAAC.pls
Reception: 100%
-----
WORC-FM "Nash Icon 98.9" Webster, MA @:
http://provisioning.streamtheworld.com/pls/WORCFMAAC.pls
Reception: 100%
-----


# Try Adelaide, Australia

34.9285° S, 138.6007° E

In [17]:
show_stations_from_coords('-34.9285+138.6007')

Found 5 cities: Adelaide, Morphett Vale, Parafield Gardens, Paralowie, Adelaide
ABC Local Radio 891 Adelaide (AAC) @:
http://live-radio01.mediahubaustralia.com/5LRW/aac/
Reception: 100%
-----
5MBS 99.9 Adelaide, SA @:
http://stream.5mbs.com/5mbs-128-mp3
Reception: 100%
-----
Triple M Adelaide @:
https://wz5web.scahw.com.au/live/5mmm_128.stream/playlist.m3u8
Reception: 100%
-----
ABC Local Radio 891 Adelaide (MP3) @:
http://live-radio01.mediahubaustralia.com/5LRW/mp3/
Reception: 100%
-----


There are 4 stations tagged as being from Adelaide, so randomly pick one.

# Try Tonga!
21.1790° S, 175.1982° W

In [19]:
show_stations_from_coords('-21.1790-175.1982')

Found 5 cities: Nuku'alofa, Haveluloto, Vaini (Tonga), ‘Ohonua, Kolonga



KeyError: 'city_name'

This time there is 1 station in the whole country, so just return that one.

# Try Buenos Aires!

34.6037° S, 58.3816° W

In [20]:
show_stations_from_coords('-34.6037-58.3816')

Found 5 cities: Buenos Aires, La Matanza Partido, La Plata Partido, Lomas de Zamora Partido, Almirante Brown

Free FM Buenos Aires @:
http://pollux.shoutca.st:8968/
Reception: 100%
-----
Mindalia Buenos Aires @:
http://188.165.131.13:7001/live
Reception: 100%
-----
LRA1 Radio Nacional Buenos Aires @:
http://sa.mp3.icecast.magma.edge-access.net:7200/sc_rad1
Reception: 100%
-----
Radio Activa FM96.9 - Colectivo Cultural @:
http://streaming01.shockmedia.com.ar:9332/stream
Reception: 100%
-----
Mindalia Voz Buenos Aires @:
http://188.165.131.13:7007/live
Reception: 100%
-----
FM del 15 - 96.1 @:
https://freeus2.listen2myradio.com/live.mp3?typeportmount=s2_37442_stream_449295080
Reception: 100%
-----
Nacional Buenos Aires - LRA1 AM870 @:
http://sa.mp3.icecast.magma.edge-access.net:7200/sc_rad1
Reception: 100%
-----
Radio Genesis AM 970 Buenos Aires @:
http://streaming.radiolinksmedia.com:7890/;
Reception: 100%
-----


# Try LA!

In [21]:
show_stations_from_coords('+34.0522-118.2437')

Found 5 cities: Los Angeles, Long Beach, Anaheim, Santa Ana, Riverside

LifeTalkRadio @:
http://ltr.streamon.fm/hls/stream/LTR-64k.aac.m3u8
Reception: 100%
-----
KKJZ "K-JAZZ 88.1" Long Beach, CA @:
http://1.ice1.firststreaming.com/kkjz_fm.aac
Reception: 100%
-----
Kill Radio - Los Angeles, CA @:
http://killradio.org:8000/killradio64.mp3.m3u
Reception: 100%
-----
KPFK 90.7 Los Angeles, CA @:
http://ic1.mainstreamnetwork.com/kpfk-fm
Reception: 100%
-----
KUCR 88.3 Riverside, CA @:
http://138.23.75.92:8001
Reception: 100%
-----
KOLA 99.9 Redlands, CA @:
http://provisioning.streamtheworld.com/pls/KOLAFMAAC.pls
Reception: 100%
-----
KSSE "La Suavecita 107.1" Arcadia, CA @:
http://provisioning.streamtheworld.com/pls/KSSEFMAAC.pls
Reception: 100%
-----
CRN 4 @:
http://64.78.234.173:8512/;
Reception: 100%
-----
KXOL "Mega" 96.3 FM Los Angeles, CA @:
http://stream.abacast.net//playlist//sbsystems-kxolfmaac-ib-64.m3u
Reception: 100%
-----
Playboy Radio @:
http://ice55.securenetsystems.net/DASH5

This isn't totally perfect, because it returns a lot of stations for this densely populated area, and one of them is technically in compton, but it's still pretty good.

# Try Ghana!

7.9465° N, 1.0232° W

In [22]:
show_stations_from_coords('+7.9465-1.0232')

Found 5 cities: Ejura, Techiman, Kintampo, Ghana, Mampong, Salaga



IndexError: list index out of range

# Try Tunis! 
33.8869° N, 9.5375° E

In [23]:
show_stations_from_coords('+36.81669331025851+10.16049291636662')

Found 5 cities: Tunis, Sousse, Bizerte, Aryanah, Hammamet

Panorama Tunis @:
https://stream.tun-radio.com/radio/8000/panorama_92
Reception: 0%
-----


# Try Seattle!

In [25]:
show_stations_from_coords('+47.6062-122.3321')

Found 5 cities: Seattle, Tacoma, Bellevue, Everett, Kent

KIRO - 710 AM ESPN Seattle @:
http://playerservices.streamtheworld.com/api/livestream-redirect/KIROAM.mp3
Reception: 100%
-----
KBFG 107.3 Seattle @:
http://192.81.134.38:8001/kbfg-1
Reception: 100%
-----
WestSeattle.Online/Radio @:
https://stream.radio.co/s80a8b3a06/listen
Reception: 100%
-----
KKNW 1150 "Alternative Talk" Seattle, WA @:
http://provisioning.streamtheworld.com/pls/KKNWAMAAC.pls
Reception: 100%
-----
KEXP 90.3 Seattle, WA @:
http://live-mp3-128.kexp.org:80/kexp128.mp3
Reception: 100%
-----
KNHC "C89.5" Seattle, WA @:
http://www.c895.org/streams/c895sc128.pls
Reception: 100%
-----
KTTH 770 Seattle, WA @:
http://playerservices.streamtheworld.com/m3u/KTTHAMAAC.m3u
Reception: 100%
-----
KEXP 90.3 Seattle, WA @:
http://live-aacplus-64.kexp.org/kexp64.aac
Reception: 100%
-----
KBCS 91.3 Bellevue College, WA @:
http://www.ophanim.net:7720/stream
Reception: 100%
-----
KXSU 102.1 FM - Seattle University’s Student-Run Radi

# Try Tokyo!

In [26]:
show_stations_from_coords('+35.68183230517904+139.76888063392605')

Found 5 cities: Tokyo, special wards of Tokyo, Yokohama, Kawasaki, Saitama

Musashino FM (むさしのFM, JOZZ3AG-FM, 78.2 MHz, Musashino City, Tokyo) @:
http://musicbird.leanstream.co/JCB032-MP3
Reception: 100%
-----
Free FM Tokyo @:
http://pollux.shoutca.st:8968/
Reception: 100%
-----
Free Fm Rock Tokyo @:
http://tachyon.shoutca.st:8298
Reception: 100%
-----
84.3 FM Edogawa (FMえどがわ, JOZZ3AS-FM, Edogawa City, Tokyo) @:
http://musicbird.leanstream.co/JCB032-MP3
Reception: 100%
-----
